### Do an exploration of other sessions, see if utils and stuff work for them

In [1]:
%load_ext autoreload
%autoreload 2

import glob
import os
from datetime import datetime
import pandas as pd
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
from lfp_tools import (
    startup as lfp_startup,
)
import json

### Grab sessions and put info in a dataframe

In [2]:
# NOTE: this is hacky code to replicate information that should already be stored in a Datajoint table, but don't have access atm
sess_folder_path = "/data/rawdata/sub-SA/"
sess_paths = glob.glob(f"{sess_folder_path}/sess-*")
session_names = [os.path.split(sess_path)[1].split("-")[1] for sess_path in sess_paths]

rows = []
for sess_name in session_names:
    if not sess_name.isdigit():
        continue
    # hacky way to grab a datetime
    date = datetime.strptime(sess_name[:8], "%Y%m%d").date()
    rest = sess_name[8:]
    count = int(rest) if rest else 0
    rows.append({
        "session_datetime": date,
        "session_count": count,
        "session_name": sess_name,
    })
sess_df = pd.DataFrame(rows)

### Filter sessions by num neurons, num trials

In [3]:
NUM_NEURONS = 20
NUM_TRIALS = 500
VALID_SESS_BEFORE = datetime.strptime("20181015", "%Y%m%d").date()

In [ ]:
def check_num_trials(sess):
    sess_name = sess.session_name
    behavior_path = f"/data/rawdata/sub-SA/sess-{sess_name}/behavior/sub-SA_sess-{sess_name}_object_features.csv"
    if not os.path.isfile(behavior_path):
        return False
    beh = pd.read_csv(behavior_path)
    valid_beh = beh[beh.Response.isin(["Correct", "Incorrect"])]   
    return len(valid_beh) > 500

def check_date(sess):
    return sess.session_datetime < VALID_SESS_BEFORE

def check_num_neurons(sess):
    spike_dir_path = f"/data/rawdata/sub-SA/sess-{sess.session_name}/spikes"
    if not os.path.isdir(spike_dir_path):
        return False
    spike_times = spike_general.get_spike_times(None, "SA", sess.session_name, species_dir="/data")
    return len(spike_times.UnitID.unique()) > NUM_NEURONS

def filter_sessions(sess):
    return check_date(sess) and check_num_trials(sess) and check_num_neurons(sess)


sess_df["valid"] = sess_df.apply(filter_sessions, axis=1)

### Find 36 Session before task change date, with at least 500 trials and 20 neurons

In [29]:
len(sess_df[sess_df.valid])

36

### Look at electrode positions again, weights

In [7]:
def get_electrode_locations(row):
    session = row.session_name
    # For the cases like 201807250001
    sess_day = session[:8]
    info_path = f"/data/rawdata/sub-SA/sess-{sess_day}/session_info/sub-SA_sess-{sess_day}_sessioninfo.json"
    with open(info_path, 'r') as f:
        data = json.load(f)
    locs = data['electrode_info']
    locs_df = pd.DataFrame.from_dict(locs)
    electrode_pos_not_nan = locs_df[~locs_df['x'].isna() & ~locs_df['y'].isna() & ~locs_df['z'].isna()]
    units = spike_general.list_session_units(None, "SA", session, species_dir="/data")
    unit_pos = pd.merge(units, electrode_pos_not_nan, left_on="Channel", right_on="electrode_id", how="left")
    unit_pos = unit_pos.astype({"UnitID": int})
    unit_pos["session"] = session
    return unit_pos

In [5]:
units = spike_general.list_session_units(None, "SA", "20180802", species_dir="/data")

In [8]:
valid_sess = pd.read_pickle("/data/patrick_scratch/multi_sess/valid_sessions.pickle")
all_sess_locations = pd.concat(valid_sess.apply(get_electrode_locations, axis=1).values, ignore_index=True)

In [9]:
all_sess_locations

,UnitID,Channel,Unit,SpikeTimesFile,electrode_id,x,y,z,distance,in_brain,tissue,structure_level1,structure_level2,structure_level3,structure_level4,structure_level5,structure_level6,structure_potential,session
0,0,31,1,/data/rawdata/sub-SA/sess-20180709/spikes/sub-...,31,4.231776,-73.619593,1.559706,8.75000,True,Cortical_gray_matter,Parietal_Lobe (Parietal),posterior_medial_cortex (PMC),posterior_cingulate_gyrus (PCgG),posterior_cingulate_cortex (PCC),area_23 (area_23),area_23c (area_23c),None,20180709
1,1,89a,1,/data/rawdata/sub-SA/sess-20180709/spikes/sub-...,89a,2.354436,-75.892485,29.306235,10.56250,True,Cortical_gray_matter,Frontal_Lobe (Frontal),motor_cortex (motor),medial_supplementary_motor_areas (SMA/preSMA),medial_supplementary_motor_areas (SMA/preSMA),presupplementary_motor_area (preSMA),presupplementary_motor_area (preSMA),None,20180709
2,2,42a,1,/data/rawdata/sub-SA/sess-20180709/spikes/sub-...,42a,11.622132,-70.948350,36.590204,7.50000,True,White_matter,None,None,None,None,None,None,None,20180709
3,3,8a,1,/data/rawdata/sub-SA/sess-20180709/spikes/sub-...,8a,-0.342060,-76.335619,39.109245,8.50000,True,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),dorsolateral_prefrontal_cortex (dlPFC),area_8B (area_8B),area_8B (area_8B),medial_area_8B (area_8Bm),dlPFC,20180709
4,4,95a,1,/data/rawdata/sub-SA/sess-20180709/spikes/sub-...,95a,2.062497,-78.563568,28.864301,8.12500,True,White_matter,None,None,None,None,None,None,None,20180709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,25,25a,1,/data/rawdata/sub-SA/sess-20180910/spikes/sub-...,25a,4.793843,-73.980388,40.538300,7.62500,True,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),dorsolateral_prefrontal_cortex (dlPFC),rostral_dorsolateral_prefrontal_cortex (rostra...,dorsal_area_46 (area_46d),dorsal_area_46 (area_46d),dlPFC,20180910
1115,26,39a,1,/data/rawdata/sub-SA/sess-20180910/spikes/sub-...,39a,11.623372,-69.652737,37.916912,7.75000,True,Cortical_gray_matter,Frontal_Lobe (Frontal),orbital_frontal_cortex (OFC),lateral_orbital_frontal_cortex (lat_OFC),medial_and_orbital_area_12 (area_12m/o),medial_and_orbital_area_12 (area_12m/o),medial_area_12 (area_12m),None,20180910
1116,27,23,1,/data/rawdata/sub-SA/sess-20180910/spikes/sub-...,23,4.223423,-70.308308,1.052348,11.90625,True,Cortical_gray_matter,Parietal_Lobe (Parietal),superior_parietal_lobule (SPL),area_5 (area_5),area_5 (area_5),area_PEa (PEa),area_PEa (PEa),None,20180910
1117,28,28a,2,/data/rawdata/sub-SA/sess-20180910/spikes/sub-...,28a,5.568581,-73.424981,41.715624,7.12500,True,Cortical_gray_matter,Frontal_Lobe (Frontal),lateral_prefrontal_cortex (lat_PFC),dorsolateral_prefrontal_cortex (dlPFC),rostral_dorsolateral_prefrontal_cortex (rostra...,dorsal_area_46 (area_46d),dorsal_area_46 (area_46d),dlPFC,20180910


In [10]:
len(all_sess_locations)

1119

In [11]:
all_sess_locations.structure_level1.unique()

array(['Parietal_Lobe (Parietal)', 'Frontal_Lobe (Frontal)', None,
       'telencephalon (tel)', 'Temporal_Lobe (Temporal)', nan,
       'diencephalon (di)', 'Occipital_Lobe (Occipital)'], dtype=object)

In [12]:
all_sess_locations.structure_level2.unique()

array(['posterior_medial_cortex (PMC)', 'motor_cortex (motor)', None,
       'lateral_prefrontal_cortex (lat_PFC)',
       'anterior_cingulate_gyrus (ACgG)',
       'lateral_and_ventral_pallium (LVPal)', 'amygdala (Amy)',
       'inferior_temporal_cortex (ITC)', 'preoptic_complex (POC)',
       'orbital_frontal_cortex (OFC)', 'basal_ganglia (BG)', nan,
       'medial_pallium (MPal)',
       'floor_of_the_lateral_sulcus (floor_of_ls)', 'thalamus (Thal)',
       'superior_parietal_lobule (SPL)', 'diagonal_subpallium (DSP)',
       'primary_visual_cortex (V1)', 'inferior_parietal_lobule (IPL)',
       'extrastriate_visual_areas_2-4 (V2-V4)',
       'medial_temporal_lobe (MTL)', 'somatosensory_cortex (SI/SII)'],
      dtype=object)

In [13]:
all_sess_locations.structure_level3.unique()

array(['posterior_cingulate_gyrus (PCgG)',
       'medial_supplementary_motor_areas (SMA/preSMA)', None,
       'dorsolateral_prefrontal_cortex (dlPFC)',
       'anterior_cingulate_cortex (ACC)', 'lateral_pallium (LPal)',
       'subpallial_amygdala (spAmy)',
       'fundus_of_the_superior_temporal_sulcus (STSf)',
       'preoptic_complex (POC)',
       'lateral_orbital_frontal_cortex (lat_OFC)', 'striatum (Str)',
       'ventrolateral_prefrontal_cortex (vlPFC)', nan,
       'hippocampal_formation (HF)', 'area_TE (TE)',
       'pallial_amygdala (pAmy)',
       'floor_of_the_lateral_sulcus (floor_of_ls)',
       'geniculate_thalamus (GThal)', 'area_5 (area_5)',
       'diagonal_subpallium (DSP)', 'primary_visual_cortex (V1)',
       'lateral_intraparietal_sulcus (lat_IPS)',
       'preoccipital_visual_areas_2-3 (V2-V3)', 'rhinal_cortex (Rh)',
       'area_7_in_the_inferior_parietal_lobule (area_7_in_IPL)',
       'medial_orbital_frontal_cortex (med_OFC)',
       'medial_superior_tempora

In [14]:
all_sess_locations.groupby(all_sess_locations.structure_level1, dropna=False).count()[["electrode_id"]].rename(columns={"electrode_id": "num_neurons"}).to_csv("/data/patrick_scratch/num_electrodes/structure_level1.csv")
all_sess_locations.groupby(all_sess_locations.structure_level2, dropna=False).count()[["electrode_id"]].rename(columns={"electrode_id": "num_neurons"}).to_csv("/data/patrick_scratch/num_electrodes/structure_level2.csv")
all_sess_locations.groupby(all_sess_locations.structure_level3, dropna=False).count()[["electrode_id"]].rename(columns={"electrode_id": "num_neurons"}).to_csv("/data/patrick_scratch/num_electrodes/structure_level3.csv")
